In [ ]:
#### IMPORTS ####
using Plots 
using Measures
using LaTeXStrings
using DelimitedFiles
# using NPZ
include("source/mOrbRG.jl")
include("source/probes.jl")
const SIZE = (750,300)
scale = 1.5

In [ ]:
function getRgFlow(size_BZ, J_val, W_by_J_range, orbitals)
    num_kspace_half = trunc(Int, (size_BZ - 1)/2)
    num_kspace = 2 * num_kspace_half + 1
    node = trunc(Int64, 0.5 * num_kspace_half * (num_kspace + 1) + 1)
    antinode = trunc(Int64, num_kspace_half + 1)
    k_vals = collect(range(K_MIN, K_MAX, length=num_kspace)) ./ pi
    results1_arr = zeros((length(W_by_J_range), num_kspace^2))
    results2_arr = zeros((length(W_by_J_range), num_kspace^2))
    results3_arr = zeros((length(W_by_J_range), num_kspace^2))
    results4_arr = zeros((length(W_by_J_range), num_kspace^2))
    for (j, W_by_J) in collect(enumerate(W_by_J_range))
        savePath = "data/$(orbitals[1])_$(num_kspace_half)_$(round(J_val, digits=3))_$(round(W_by_J, digits=3))"
        kondoJArray, dispersionArray = main(num_kspace_half, J_val, J_val * W_by_J, orbitals[1], orbitals[2])

        results_norm, results_unnorm, results_bool = totalScatterProb(kondoJArray, num_kspace_half+1, savePath)
        results1_arr[j,:] = results_bool
        results2_arr[j,:] = results_norm
        results3_arr[j,:] = kondoJArray[node, :, end]
        results4_arr[j,:] = kondoJArray[antinode, :, end]
    end
    max_r = maximum(log10.(results2_arr))
    min_r = minimum(log10.(results2_arr))
    pdfFileNames = []
    anim = @animate for (j, W_by_J) in enumerate(W_by_J_range)
        hm1 = heatmap(k_vals, k_vals, reshape(results1_arr[j,:], (num_kspace, num_kspace)), 
            cmap=discreteCgrad)
        hm2 = heatmap(k_vals, k_vals, reshape(log10.(results2_arr[j,:]), (num_kspace, num_kspace)),
            cmap=:matter, clims=(min_r, max_r))
        p = plot(hm1, hm2, size=SIZE, 
            plot_title="\$W/J=$(round(W_by_J, digits=3))\$", 
            top_margin=3mm, left_margin=[5mm 10mm], bottom_margin=5mm,
            xlabel="\$ \\mathrm{k_x/\\pi} \$", ylabel="\$ \\mathrm{k_y/\\pi} \$",
            dpi=100 * scale
        )
        pdfFileName = "fig_$(j).pdf"
        savefig(pdfFileName)
        push!(pdfFileNames, pdfFileName)
    end
    run(`pdfunite $pdfFileNames fig_combined.pdf`)
    run(`rm $pdfFileNames`)
    gif(anim, animName(orbitals[1], num_kspace, scale, 1) * ".gif", fps=0.5)
    gif(anim, animName(orbitals[1], num_kspace, scale, 1) * ".mp4", fps=0.5)
    
    pdfFileNames = []
    anim = @animate for (j, W_by_J) in enumerate(W_by_J_range)
        hm1 = heatmap(k_vals, k_vals, reshape(results3_arr[j,:], (num_kspace, num_kspace)), 
            cmap=:matter)
        hm2 = heatmap(k_vals, k_vals, reshape(results4_arr[j,:], (num_kspace, num_kspace)),
            cmap=:matter)
        scatter!(hm1, [-1/2], [-1/2], markersize=6, markercolor=:grey, markerstrokecolor=:white, markerstrokewidth=4)
        scatter!(hm2, [0], [-1], markersize=6, markercolor=:grey, markerstrokecolor=:white, markerstrokewidth=4)
        p = plot(hm1, hm2, size=SIZE, 
            plot_title="\$W/J=$(round(W_by_J, digits=3))\$", 
            top_margin=3mm, left_margin=[5mm 10mm], bottom_margin=5mm,
            xlabel="\$ \\mathrm{k_x/\\pi} \$", ylabel="\$ \\mathrm{k_y/\\pi} \$",
            dpi=100 * scale, legend=false
        )
        pdfFileName = "fig_$(j).pdf"
        savefig(pdfFileName)
        push!(pdfFileNames, pdfFileName)
    end
    run(`pdfunite $pdfFileNames fig_combined_nesting.pdf`)
    run(`rm $pdfFileNames`)
    gif(anim, animName(orbitals[1], num_kspace, scale, 1) * "_nesting.gif", fps=0.5)
    gif(anim, animName(orbitals[1], num_kspace, scale, 1) * "_nesting.mp4", fps=0.5);
end

# getRgFlow(29, 1., 0:-0.05:-0.2, ["p", "p"])
manager(43, 0.1, collect(0:-0.02:-0.12), ["p", "p"], ["scattProb", "kondoCoupFSMap", "kondoCoupDiagMap"], 2.)